In [ ]:
from pathlib import Path

import spiceypy as spice

from planetarypy.spice import (
    datasets,
    download_generic_kernels,
    get_metakernel_and_files,
    list_kernels_for_day,
    load_generic_kernels,
)

In [19]:
def last_part(path, n):
    """Show only the last n parts from a pathlib.Path object."""
    # this is useful to remove machine-specific leading paths
    # and keep the notebook clean even when coded on different machines
    return Path(*path.parts[-n:]) if n > 0 else path

# NAIF Archived SPICE Kernels
Below table shows all officially archived SPICE kernels from NAIF. 
These kernels are less often updated and are not guaranteed to be the most recent versions.

However, they are still useful for many applications and can be used as a stable reference.

Some ongoing missions like MRO and LRO regularly update their archived kernels, but not very frequently.

The `datasets` object is just a parsed table from the NAIF website [here](https://naif.jpl.nasa.gov/naif/data_archived.html).

In [14]:
datasets

,path,Mission Name,PDS3 or PDS4,Data Size (GB),Start Time,Stop Time
shorthand,,,,,,
bc,bc/bc_spice,BepiColombo,4,2.6,2018-10-20,2024-01-01
cassini,co-s_j_e_v-spice-6-v1.0/cosp_1000,Cassini Orbiter,3,73.5,1997-10-15,2017-09-15
clementine,clem1-l-spice-6-v1.0/clsp_1000,Clementine,3,0.8,1994-01-26,1994-05-07
dart,dart/dart_spice,DART,4,10.1,2021-11-09,2050-01-01
dawn,dawn-m_a-spice-6-v1.0/dawnsp_1000,DAWN,3,86.4,2007-09-27,2018-10-31
di,di-c-spice-6-v1.0/disp_1000,Deep Impact,3,0.7,2005-01-12,2005-08-09
ds1,ds1-a_c-spice-6-v1.0/ds1sp_1000,Deep Space 1,3,0.9,1998-10-24,2001-12-18
epoxi,dif-c_e_x-spice-6-v1.0/epxsp_1000,EPOXI,3,1.0,2005-08-23,2011-03-01
em16,em16/em16_spice,ExoMars TGO 2016,4,11.0,2016-03-14,2024-01-01


In [15]:
list_kernels_for_day?

Signature: list_kernels_for_day(mission: str, start: str, stop: str = '') -> list
Docstring:
List all kernels for a given time range of a mission.

Parameters
----------
mission : str
    Mission shorthand in datasets dataframe.
start : str
    Start time in either ISO or yyyy-jjj format.
stop : str, optional
    Stop time in either ISO or yyyy-jjj format. Defaults to None.
File:      ~/Library/CloudStorage/Dropbox/src/planetarypy/src/planetarypy/spice/archived_kernels.py
Type:      function

In [16]:
list_kernels_for_day("messenger", "2011-01-01")

['ck/msgr_1101_v02.bc',
 'ck/msgr_mdis_gm040819_150430v1.bc',
 'fk/msgr_dyn_v600.tf',
 'fk/msgr_v231.tf',
 'ik/msgr_epps_v100.ti',
 'ik/msgr_grns_v110.ti',
 'ik/msgr_mag_v021.ti',
 'ik/msgr_mascs_v100.ti',
 'ik/msgr_mdis_v160.ti',
 'ik/msgr_mla_v010.ti',
 'ik/msgr_rs_v111.ti',
 'ik/msgr_xrs_v001.ti',
 'lsk/naif0011.tls',
 'pck/pck00010_msgr_v23.tpc',
 'sclk/messenger_2548.tsc',
 'spk/msgr_040803_150430_150430_od431sc_2.bsp',
 'spk/msgr_antenna_v000.bsp']

## Most useful function `get_metakernel_and_files()`:

For a given mission and time-range, it downloads and stores all relevant archived kernels in a local archive, edits the provided meta-kernel from NAIF to point to the correct local paths, and returns the path to the local meta-kernel, ready to be loaded with spiceypy.furnsh().

In [20]:
mk_path = get_metakernel_and_files(
    mission="cassini", start="2015-01-01", stop="2015-01-02"
)
last_part(mk_path, 3)

Kernels downloaded:   0%|          | 0/31 [00:00<?, ?it/s]

PosixPath('spice_kernels/cassini/cas_2015_v09_150101_150102.tm')

As you can see, the meta-kernel can be loaded immediately with spiceypy:

In [25]:
spice.furnsh(str(mk_path))  # spiceypy requires a string path

### Generic Kernels
The below functions download and load a few generic kernels that enable some basic SPICE calculations with the major planetary bodies.

TODO: Script a test that shows for which bodies some basic calculations can be done.

In [26]:
load_generic_kernels()  # uses below download_generic_kernels if any kernel from generic list is missing.

In [27]:
download_generic_kernels()

naif0012.tls already downloaded. Use `overwrite=True` to download again.
pck00010.tpc already downloaded. Use `overwrite=True` to download again.
de-403-masses.tpc already downloaded. Use `overwrite=True` to download again.
de430.bsp already downloaded. Use `overwrite=True` to download again.
mar097.bsp already downloaded. Use `overwrite=True` to download again.
